In [12]:
import tvm
from tvm import te
import numpy as np

In [13]:
M = 2048
K = 2048
bn = 32

dtype = "int32"
target = tvm.target.Target(target="upmem", host="llvm")

In [14]:
k = te.reduce_axis((0, K), "k")
A = te.placeholder((M, K), dtype, "A")
B = te.placeholder((K,), dtype, "B")
C = te.compute((M,), lambda y: te.sum(A[y, k] * B[k], axis=k), name="C")

s = te.create_schedule(C.op)

In [15]:
xb, _ = s[C].split(k, nparts = 4)
CF = s.rfactor(C, xb)

In [16]:
AL = s.cache_read(A, "local", [CF])
BL = s.cache_read(B, "local", [CF])
CL = s.cache_write(CF, "local")

In [17]:
xb, yb =s[CF].op.axis

In [18]:
yb, yo = s[CF].split(yb, nparts=4)
yo, yi = s[CF].split(yo, nparts=16)
yi, yc = s[CF].split(yi, 2)
s[CF].reorder(yo, yi, yc)

In [19]:
s[CF].bind(yb, te.thread_axis("blockIdx.x"))
s[CF].bind(xb, te.thread_axis("blockIdx.y"))
s[CF].bind(yo, te.thread_axis("threadIdx.x"))

In [22]:
s[CL].compute_at(s[CF], yi)
xo, xi = s[CL].split(s[CL].op.reduce_axis[0], 64)
s[CL].reorder(s[CL].op.axis[0], xo, xi)
s[AL].compute_at(s[CL], xo)
s[BL].compute_at(s[CL], xo)

In [23]:
print(tvm.lower(s, [A, B, C], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "int32"), B: T.Buffer((2048,), "int32"), C: T.Buffer((2048,), "int32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "global_symbol": "main", "tir.noalias": T.bool(True)})
        C_rf = T.allocate([8192], "int32", "global")
        C_rf_1 = T.Buffer((8192,), "int32", data=C_rf)
        with T.launch_thread("blockIdx.y", 4) as blockIdx_y:
            C_rf_local = T.allocate([2], "int32", "local")
            A_local = T.allocate([64], "int32", "local")
            B_local = T.allocate([64], "int32", "local")
            blockIdx_x = T.launch_thread("blockIdx.x", 4)
            threadIdx_x = T.launch_thread("threadIdx.x", 16)
            for y_inner_inner_outer in range(16):
                C_rf_local_1 = T.Buffer((2,), "int32", data=C_rf_local, scope="local", align=8)
                for y_c in range(2):
               

In [24]:
func = tvm.build(s, [A, B, C], target=target, name="mmult")

In [25]:
func

Module(llvm, 325a1f8)